In [1]:
using JuMP, Ipopt, Plots
default(size=(1000,600))

In [2]:
dx=0.01
dy=0.01

grid = [[x;y] for x in -1:dx:1-dx, y in -1:dx:1-dy]

200×200 Matrix{Vector{Float64}}:
 [-1.0, -1.0]   [-1.0, -0.99]   …  [-1.0, 0.98]   [-1.0, 0.99]
 [-0.99, -1.0]  [-0.99, -0.99]     [-0.99, 0.98]  [-0.99, 0.99]
 [-0.98, -1.0]  [-0.98, -0.99]     [-0.98, 0.98]  [-0.98, 0.99]
 [-0.97, -1.0]  [-0.97, -0.99]     [-0.97, 0.98]  [-0.97, 0.99]
 [-0.96, -1.0]  [-0.96, -0.99]     [-0.96, 0.98]  [-0.96, 0.99]
 [-0.95, -1.0]  [-0.95, -0.99]  …  [-0.95, 0.98]  [-0.95, 0.99]
 [-0.94, -1.0]  [-0.94, -0.99]     [-0.94, 0.98]  [-0.94, 0.99]
 [-0.93, -1.0]  [-0.93, -0.99]     [-0.93, 0.98]  [-0.93, 0.99]
 [-0.92, -1.0]  [-0.92, -0.99]     [-0.92, 0.98]  [-0.92, 0.99]
 [-0.91, -1.0]  [-0.91, -0.99]     [-0.91, 0.98]  [-0.91, 0.99]
 [-0.9, -1.0]   [-0.9, -0.99]   …  [-0.9, 0.98]   [-0.9, 0.99]
 [-0.89, -1.0]  [-0.89, -0.99]     [-0.89, 0.98]  [-0.89, 0.99]
 [-0.88, -1.0]  [-0.88, -0.99]     [-0.88, 0.98]  [-0.88, 0.99]
 ⋮                              ⋱                 
 [0.88, -1.0]   [0.88, -0.99]      [0.88, 0.98]   [0.88, 0.99]
 [0.89, -1.0]   [0.89, 

In [3]:
function integrate2(h,grid,dx,dy)
    return sum(h.(grid)*dx*dy)
end

integrate2 (generic function with 1 method)

In [4]:
#h raised cos
h(x,y)=1+cos(2*pi*x)*cos(2*pi*y)
h(v)=h(v...)
integrate2(h,grid,dx,dy)

4.0

In [5]:
#h uniforme
h(x,y)=0.25
h(v)=h(v...)
integrate2(h,grid,dx,dy)

1.0

In [6]:
## Optimizo placement

n=2

model = JuMP.Model(Ipopt.Optimizer)

@variable(model,-1<=y[1:n,1:2]<=1)

expr=[];

for j=1:n
    for i=1:length(grid[:])
        name=Symbol("d_$(i)$(j)")
        ####falta agregar sqrt sin que de problemas numericos!!
        register(model, name, 2, (y1,y2)->sqrt(.001+(grid[i][1]-y1)^2+(grid[i][2]-y2)^2) ,autodiff = true)
        expr_i = add_nonlinear_expression(model, :($name($(y[j,1]),$(y[j,2]))))
        push!(expr,expr_i)
    end
end
@NLobjective(model,Min,sum(expr_i for expr_i in expr))

#print(model)
#expr

In [7]:
optimize!(model)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        4
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        4
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality co

In [8]:
solution_summary(model)

* Solver : Ipopt

* Status
  Termination status : LOCALLY_SOLVED
  Primal status      : FEASIBLE_POINT
  Dual status        : FEASIBLE_POINT
  Message from the solver:
  "Solve_Succeeded"

* Candidate solution
  Objective value      : 6.12843e+04
  Dual objective value : -7.04832e-13

* Work counters
  Solve time (sec)   : 3.86605e+00


In [9]:
value.(y)

2×2 Matrix{Float64}:
 -0.005  -0.005
 -0.005  -0.005